# 実装の設計
> *どのように実装すれば効率よくディープニューラルネットモデルを定義できるか*
## 基本設計
### TensorFlowによる実装
<div style="text-align:center">
    <b>モデルの出力を定義</b><br>
    ↓<br>
    <b>誤差関数を定義</b><br>
    ↓<br>
    <b>モデルを学習</b>
</div>
TensorFlowでは、上記の流れをステップごとに関数化することを推奨している。

> - `inference()` : モデル全体の設定を行い、モデルの出力・予測結果を返す
> - `loss()` : モデルの誤差関数を定義し、誤差・損失を返す
> - `training()` : モデルの学習を行い、学習結果(進み具合)を返す

```python
def inference(x):
    """
    モデルの定義
    """
    
def loss(pred_y, y):
    """
    誤差関数の定義
    
    Parameter
    - pred_y : 予測値
    - y : 正解データ
    """

def training(loss):
    """
    学習アルゴリズムの定義
    """

if __name__ == '__main__':
    # 1. データの準備
    # 2. モデル設定
    model = inference(x)
    loss = loss(model, y)
    train_step = training(loss)
    # 3. モデル学習
    # 4. モデル評価
```

In [1]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

np.random.seed(0)
tf.set_random_seed(1234)


def inference(x, keep_prob, n_in, n_hiddens, n_out):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.zeros(shape)
        return tf.Variable(initial)

    # 入力層 - 隠れ層、隠れ層 - 隠れ層
    for i, n_hidden in enumerate(n_hiddens):
        if i == 0:
            input = x
            input_dim = n_in
        else:
            input = output
            input_dim = n_hiddens[i-1]

        W = weight_variable([input_dim, n_hidden])
        b = bias_variable([n_hidden])

        h = tf.nn.relu(tf.matmul(input, W) + b)
        output = tf.nn.dropout(h, keep_prob)

    # 隠れ層 - 出力層
    W_out = weight_variable([n_hiddens[-1], n_out])
    b_out = bias_variable([n_out])
    y = tf.nn.softmax(tf.matmul(output, W_out) + b_out)
    return y


def loss(pred_y, y):
    cross_entropy = \
        tf.reduce_mean(-tf.reduce_sum(
                       y * tf.log(tf.clip_by_value(pred_y, 1e-10, 1.0)),
                       axis=1))
    return cross_entropy


def training(loss):
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train_step = optimizer.minimize(loss)
    return train_step


def accuracy(y, t):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy


if __name__ == '__main__':
    '''
    1. データの準備
    '''
    mnist = datasets.fetch_mldata('MNIST original', data_home='.')

    n = len(mnist.data)
    N = 30000  # MNISTの一部を使う
    N_train = 20000
    N_validation = 4000
    indices = np.random.permutation(range(n))[:N]  # ランダムにN枚を選択

    X = mnist.data[indices]
    y = mnist.target[indices]
    Y = np.eye(10)[y.astype(int)]  # 1-of-K 表現に変換

    X_train, X_test, Y_train, Y_test = \
        train_test_split(X, Y, train_size=N_train)

    X_train, X_validation, Y_train, Y_validation = \
        train_test_split(X_train, Y_train, test_size=N_validation)

    '''
    2. モデル設定
    '''
    n_in = len(X[0])
    n_hiddens = [200, 200, 200]  # 各隠れ層の次元数
    n_out = len(Y[0])
    p_keep = 0.5

    x = tf.placeholder(tf.float32, shape=[None, n_in])
    t = tf.placeholder(tf.float32, shape=[None, n_out])
    keep_prob = tf.placeholder(tf.float32)

    model = inference(x, keep_prob, n_in=n_in, n_hiddens=n_hiddens, n_out=n_out)
    loss = loss(model, t)
    train_step = training(loss)

    accuracy = accuracy(model, t)

    history = {
        'val_loss': [],
        'val_acc': []
    }

    '''
    3. モデル学習
    '''
    epochs = 50
    batch_size = 200

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

    n_batches = N_train // batch_size

    for epoch in range(epochs):
        X_, Y_ = shuffle(X_train, Y_train)

        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size

            sess.run(train_step, feed_dict={
                x: X_[start:end],
                t: Y_[start:end],
                keep_prob: p_keep
            })

        # 検証データを用いた評価
        val_loss = loss.eval(session=sess, feed_dict={
            x: X_validation,
            t: Y_validation,
            keep_prob: 1.0
        })
        val_acc = accuracy.eval(session=sess, feed_dict={
            x: X_validation,
            t: Y_validation,
            keep_prob: 1.0
        })

        # 検証データに対する学習の進み具合を記録
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print('epoch:', epoch,
              ' validation loss:', val_loss,
              ' validation accuracy:', val_acc)
        
    '''
    予測精度の評価
    '''
    accuracy_rate = accuracy.eval(session=sess, feed_dict={
        x: X_test,
        y: Y_test,
        keep_prob: 1.0
    })
    print('accuracy: ', accuracy_rate)

/Users/taiyou/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/taiyou/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/taiyou/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


epoch: 0  validation loss: 2.1558669  validation accuracy: 0.26725
epoch: 1  validation loss: 0.8131207  validation accuracy: 0.768
epoch: 2  validation loss: 0.51363796  validation accuracy: 0.85125
epoch: 3  validation loss: 0.40519613  validation accuracy: 0.88725
epoch: 4  validation loss: 0.3408817  validation accuracy: 0.90225
epoch: 5  validation loss: 0.30277404  validation accuracy: 0.914
epoch: 6  validation loss: 0.26965433  validation accuracy: 0.92175
epoch: 7  validation loss: 0.25186813  validation accuracy: 0.9305
epoch: 8  validation loss: 0.23474042  validation accuracy: 0.93325
epoch: 9  validation loss: 0.22533228  validation accuracy: 0.9335
epoch: 10  validation loss: 0.21379852  validation accuracy: 0.93875
epoch: 11  validation loss: 0.20155106  validation accuracy: 0.94375
epoch: 12  validation loss: 0.20540118  validation accuracy: 0.94075
epoch: 13  validation loss: 0.18442798  validation accuracy: 0.948
epoch: 14  validation loss: 0.19221994  validation accu

TypeError: unhashable type: 'numpy.ndarray'

### Kerasによる実装

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras import backend as K
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

np.random.seed(123)

'''
1. データの生成
'''
mnist = datasets.fetch_mldata('MNIST original', data_home='.')

n = len(mnist.data)
N = 30000  # MNISTの一部を使う
N_train = 20000
N_validation = 4000
indices = np.random.permutation(range(n))[:N]  # ランダムにN枚を選択

X = mnist.data[indices]
y = mnist.target[indices]
Y = np.eye(10)[y.astype(int)]

X_train, X_test, Y_train, Y_test = \
    train_test_split(X, Y, train_size=N_train)
X_train, X_validation, Y_train, Y_validation = \
    train_test_split(X_train, Y_train, test_size=N_validation)

'''
2. モデル設定
'''
n_in = len(X[0])  # 784
n_hiddens = [200, 200, 200]
n_out = len(Y[0])  # 10
p_keep = 0.5
activation = 'relu'


def weight_variable(shape):
    return K.truncated_normal(shape, stddev=0.01)
    # return np.random.normal(scale=0.01, size=shape)


model = Sequential()
for i, input_dim in enumerate(([n_in] + n_hiddens)[:-1]):
    model.add(Dense(n_hiddens[i], input_dim=input_dim,
                    kernel_initializer=weight_variable))
    model.add(Activation(activation))
    model.add(Dropout(p_keep))

model.add(Dense(n_out, kernel_initializer=weight_variable))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01),
              metrics=['accuracy'])

'''
3. モデル学習
'''
epochs = 50
batch_size = 200

hist = model.fit(X_train, Y_train, epochs=epochs,
                 batch_size=batch_size,
                 validation_data=(X_validation, Y_validation))


'''
4. 予測精度の評価
'''
loss_and_metrics = model.evaluate(X_test, Y_test)
print(loss_and_metrics)

Using TensorFlow backend.
/Users/taiyou/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train on 16000 samples, validate on 4000 samples
Epoch 1/50
16000/16000 [==============================] - 1s 92us/step - loss: 2.2604 - acc: 0.1547 - val_loss: 2.0656 - val_acc: 0.4352
Epoch 2/50
16000/16000 [==============================] - 1s 63us/step - loss: 1.4294 - acc: 0.5261 - val_loss: 0.7200 - val_acc: 0.7723
Epoch 3/50
16000/16000 [==============================] - 1s 73us/step - loss: 0.8072 - acc: 0.7357 - val_loss: 0.4901 - val_acc: 0.8595
Epoch 4/50
16000/16000 [==============================] - 1s 58us/step - loss: 0.6132 - acc: 0.8106 - val_loss: 0.3981 - val_acc: 0.8840
Epoch 5/50
16000/16000 [==============================] - 1s 65us/step - loss: 0.5120 - acc: 0.8460 - val_loss: 0.3326 - val_acc: 0.9048
Epoch 6/50
16000/16000 [==============================] - 1s 56us/step - loss: 0.4366 - acc: 0.8721 - val_loss: 0.3011 - val_acc: 0.9110
Epoch 7/50
16000/16000 [==============================] - 1s 49us/step - loss: 0.3898 - acc: 0.8868 - val_loss: 0.2590 - val_acc:

### TensorFlowにおけるモデルのクラス化
Keras, sklearnのようなシンプルな記述をしたい。
```python
model = DNN()   # Deep Neural Net
model.fit(X_train, Y_train)
model.evaluate(X_test, Y_test)
```

In [3]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

np.random.seed(0)
tf.set_random_seed(1234)


class DNN(object):
    def __init__(self, n_in, n_hiddens, n_out):
        self.n_in = n_in
        self.n_hiddens = n_hiddens
        self.n_out = n_out
        self.weights = []
        self.biases = []

        self._x = None
        self._y = None
        self._t = None,
        self._keep_prob = None
        self._sess = None
        self._history = {
            'accuracy': [],
            'loss': []
        }

    def weight_variable(self, shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)

    def bias_variable(self, shape):
        initial = tf.zeros(shape)
        return tf.Variable(initial)

    def inference(self, x, keep_prob):
        """
        設定されたモデルを返す.
        """
        # 入力層 - 隠れ層、隠れ層 - 隠れ層
        for i, n_hidden in enumerate(self.n_hiddens):
            if i == 0:
                input = x
                input_dim = self.n_in
            else:
                input = output
                input_dim = self.n_hiddens[i-1]

            self.weights.append(self.weight_variable([input_dim, n_hidden]))
            self.biases.append(self.bias_variable([n_hidden]))

            h = tf.nn.relu(tf.matmul(
                input, self.weights[-1]) + self.biases[-1])
            output = tf.nn.dropout(h, keep_prob)

        # 隠れ層 - 出力層
        self.weights.append(
            self.weight_variable([self.n_hiddens[-1], self.n_out]))
        self.biases.append(self.bias_variable([self.n_out]))

        y = tf.nn.softmax(tf.matmul(
            output, self.weights[-1]) + self.biases[-1])
        return y

    def loss(self, y, t):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y), axis=1))
        return cross_entropy

    def training(self, loss):
        optimizer = tf.train.GradientDescentOptimizer(0.01)
        train_step = optimizer.minimize(loss)
        return train_step

    def accuracy(self, y, t):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return accuracy

    def fit(self, X_train, Y_train,
            nb_epoch=100, batch_size=100, p_keep=0.5,
            verbose=1):
        x = tf.placeholder(tf.float32, shape=[None, self.n_in])
        t = tf.placeholder(tf.float32, shape=[None, self.n_out])
        keep_prob = tf.placeholder(tf.float32)

        self._x = x
        self._t = t
        self._keep_prob = keep_prob

        y = self.inference(x, keep_prob)
        loss = self.loss(y, t)
        train_step = self.training(loss)
        accuracy = self.accuracy(y, t)

        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)

        self._y = y
        self._sess = sess

        N_train = len(X_train)
        n_batches = N_train // batch_size

        for epoch in range(nb_epoch):
            X_, Y_ = shuffle(X_train, Y_train)

            for i in range(n_batches):
                start = i * batch_size
                end = start + batch_size

                sess.run(train_step, feed_dict={
                    x: X_[start:end],
                    t: Y_[start:end],
                    keep_prob: p_keep
                })
            loss_ = loss.eval(session=sess, feed_dict={
                x: X_train,
                t: Y_train,
                keep_prob: 1.0
            })
            accuracy_ = accuracy.eval(session=sess, feed_dict={
                x: X_train,
                t: Y_train,
                keep_prob: 1.0
            })
            self._history['loss'].append(loss_)
            self._history['accuracy'].append(accuracy_)

            if verbose:
                print('epoch:', epoch,
                      ' loss:', loss_,
                      ' accuracy:', accuracy_)

        return self._history

    def evaluate(self, X_test, Y_test):
        accuracy = self.accuracy(self._y, self._t)
        return accuracy.eval(session=self._sess, feed_dict={
            self._x: X_test,
            self._t: Y_test,
            self._keep_prob: 1.0
        })


if __name__ == '__main__':
    '''
    データの生成
    '''
    mnist = datasets.fetch_mldata('MNIST original', data_home='.')

    n = len(mnist.data)
    N = 10000  # MNISTの一部を使う
    indices = np.random.permutation(range(n))[:N]  # ランダムにN枚を選択

    X = mnist.data[indices]
    y = mnist.target[indices]
    Y = np.eye(10)[y.astype(int)]  # 1-of-K 表現に変換

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

    '''
    モデル設定
    '''
    model = DNN(n_in=len(X[0]),
                n_hiddens=[200, 200, 200],
                n_out=len(Y[0]))

    '''
    モデル学習
    '''
    model.fit(X_train, Y_train,
              nb_epoch=30,
              batch_size=200,
              p_keep=0.5)

    '''
    予測精度の評価
    '''
    accuracy = model.evaluate(X_test, Y_test)
    print('accuracy: ', accuracy)

/Users/taiyou/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


epoch: 0  loss: 2.2689435  accuracy: 0.0955
epoch: 1  loss: 2.066787  accuracy: 0.32625
epoch: 2  loss: 1.3807827  accuracy: 0.636
epoch: 3  loss: 0.8205668  accuracy: 0.739625
epoch: 4  loss: 0.60315645  accuracy: 0.84125
epoch: 5  loss: 0.4939494  accuracy: 0.853875
epoch: 6  loss: 0.41529793  accuracy: 0.8785
epoch: 7  loss: 0.35941705  accuracy: 0.8955
epoch: 8  loss: 0.31095868  accuracy: 0.909875
epoch: 9  loss: 0.284324  accuracy: 0.919
epoch: 10  loss: 0.2689737  accuracy: 0.92225
epoch: 11  loss: 0.23972338  accuracy: 0.931625
epoch: 12  loss: 0.22270766  accuracy: 0.934875
epoch: 13  loss: 0.2149805  accuracy: 0.938
epoch: 14  loss: 0.18783116  accuracy: 0.9455
epoch: 15  loss: 0.17952396  accuracy: 0.947125
epoch: 16  loss: 0.16210172  accuracy: 0.952625
epoch: 17  loss: 0.15420085  accuracy: 0.95475
epoch: 18  loss: 0.13554727  accuracy: 0.961125
epoch: 19  loss: 0.13559642  accuracy: 0.958875
epoch: 20  loss: 0.12772313  accuracy: 0.96275
epoch: 21  loss: 0.11827733  accur